In [1]:
# %load https://gist.githubusercontent.com/kidpixo/2ec078d09834b5aa7869/raw/350f79fe4f2e09592404a76db18dcc01a961444b/ipython_inizialization.py
import pandas as pd
import glob

In [2]:
# delete variable only if defined
try:
    presidential # does a exist in the current namespace
except NameError:
    print 'presidential does not exist - ok'
else:
    del presidential
    print 'presidential does exist - deleting'
print 20*'='    

# cycle on all the files and extract
for state in glob.glob('nytimes_elections_results_states/*html'):
    # only the state name
    print state,

    try:
        data = pd.read_html(state,attrs = {'class': 'eln-county-table'})
    except ValueError: 
        print '- data lines 0'
        pass
    else :

        # temporary DataFrames
        tmp = data[0]
        tmp.dropna(axis=0, how='all',inplace=True) # drop empty rows
        tmp['State'] = state[33:-50] # add state

        # check if the DataFrame already exist
        try:
            presidential # does a exist in the current namespace
        except NameError:
            presidential = tmp # define it
        else:
            presidential = presidential.append(tmp) # manipulate it
        del tmp
        print '- data lines {}'.format(presidential.shape[0])        

presidential does not exist - ok
nytimes_elections_results_states/Alabama Election Results 2016 – The New York Times.html - data lines 67
nytimes_elections_results_states/Alaska Election Results 2016 – The New York Times.html - data lines 0
- data lines 67
nytimes_elections_results_states/Arizona Election Results 2016 – The New York Times.html - data lines 82
nytimes_elections_results_states/Arkansas Election Results 2016 – The New York Times.html - data lines 157
nytimes_elections_results_states/California Election Results 2016 – The New York Times.html - data lines 215
nytimes_elections_results_states/Colorado Election Results 2016 – The New York Times.html - data lines 279
nytimes_elections_results_states/Connecticut Election Results 2016 – The New York Times.html - data lines 448
nytimes_elections_results_states/Delaware Election Results 2016 – The New York Times.html - data lines 451
nytimes_elections_results_states/District of Columbia Election Results 2016 – The New York Times.h

In [3]:
len(presidential.State.unique())

49

In [4]:
# use sane columns names
presidential.columns = [l.strip(' \t\n\r.') for l in presidential.columns]
# strip % character
presidential['Rpt'] = presidential['Rpt'].apply( lambda x: x[0:-1])
# regenerate index
presidential.index = range(presidential.shape[0])

In [5]:
# some page use town some county, aggregate in column Place
presidential['Place'] = ''
presidential['Place'].loc[presidential['Vote by town'].dropna().index] = presidential['Vote by town'].dropna()
presidential['Place'].loc[presidential['Vote by county'].dropna().index] = presidential['Vote by county'].dropna()

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
presidential.to_csv('nytimes_presidential_elections_2016_results_county.csv', index=False)